In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import librosa, torch, torchaudio, joblib
import numpy as np
import scipy.io as sio
from utils import STFTToWav, wavToSTFT, makeGenerator
import os, pickle, pdb
from concurrent.futures import ThreadPoolExecutor
from params import get_arg
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
config = get_arg(['--nfft','2'])
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [9]:
trainloader = makeGenerator(torch.arange(100000).unsqueeze(-1), torch.arange(100000).unsqueeze(-1), config=config,device=device)

In [14]:
da = next(trainloader.next_loader())
for k,l in da:
    print(k.shape, l.shape)
    print(k[:10],l[:10])

torch.Size([16, 1, 400]) torch.Size([16, 200, 1])
tensor([[[  0,   1,   2,  ..., 397, 398, 399]],

        [[  0,   1,   2,  ..., 397, 398, 399]],

        [[  0,   1,   2,  ..., 397, 398, 399]],

        ...,

        [[  0,   1,   2,  ..., 397, 398, 399]],

        [[  0,   1,   2,  ..., 397, 398, 399]],

        [[  1,   2,   3,  ..., 398, 399, 400]]]) tensor([[[424],
         [425],
         [426],
         ...,
         [621],
         [622],
         [623]],

        [[424],
         [425],
         [426],
         ...,
         [621],
         [622],
         [623]],

        [[424],
         [425],
         [426],
         ...,
         [621],
         [622],
         [623]],

        ...,

        [[424],
         [425],
         [426],
         ...,
         [621],
         [622],
         [623]],

        [[424],
         [425],
         [426],
         ...,
         [621],
         [622],
         [623]],

        [[425],
         [426],
         [427],
         ...,
      

        [[136, 137, 138,  ..., 533, 534, 535]]]) tensor([[[441],
         [442],
         [443],
         ...,
         [638],
         [639],
         [640]],

        [[424],
         [425],
         [426],
         ...,
         [621],
         [622],
         [623]],

        [[448],
         [449],
         [450],
         ...,
         [645],
         [646],
         [647]],

        ...,

        [[426],
         [427],
         [428],
         ...,
         [623],
         [624],
         [625]],

        [[440],
         [441],
         [442],
         ...,
         [637],
         [638],
         [639]],

        [[560],
         [561],
         [562],
         ...,
         [757],
         [758],
         [759]]])
torch.Size([16, 1, 400]) torch.Size([16, 200, 1])
tensor([[[  2,   3,   4,  ..., 399, 400, 401]],

        [[  0,   1,   2,  ..., 397, 398, 399]],

        [[  2,   3,   4,  ..., 399, 400, 401]],

        ...,

        [[  9,  10,  11,  ..., 406, 407, 408]],

     

        [[  1,   2,   3,  ..., 398, 399, 400]]]) tensor([[[433],
         [434],
         [435],
         ...,
         [630],
         [631],
         [632]],

        [[426],
         [427],
         [428],
         ...,
         [623],
         [624],
         [625]],

        [[424],
         [425],
         [426],
         ...,
         [621],
         [622],
         [623]],

        ...,

        [[440],
         [441],
         [442],
         ...,
         [637],
         [638],
         [639]],

        [[470],
         [471],
         [472],
         ...,
         [667],
         [668],
         [669]],

        [[425],
         [426],
         [427],
         ...,
         [622],
         [623],
         [624]]])
torch.Size([16, 1, 400]) torch.Size([16, 200, 1])
tensor([[[  5,   6,   7,  ..., 402, 403, 404]],

        [[ 25,  26,  27,  ..., 422, 423, 424]],

        [[  0,   1,   2,  ..., 397, 398, 399]],

        ...,

        [[  5,   6,   7,  ..., 402, 403, 404]],

     

In [7]:

n_dims = 
stft = wavToSTFT(config, device)
istft = STFTToWav(config, device)

In [33]:
data_path = '.'
transfer_f = np.array(pickle.load(open(os.path.join(data_path,'transfer_f.pickle'),'rb')))
transfer_f = torch.from_numpy(transfer_f).to(device)
transfer_f.requires_grad = False
if config.feature in ['wav', 'mel', 'stft']:
    accel_raw_data = joblib.load(open(os.path.join(data_path,'stationary_accel_train.joblib'),'rb'))
    sound_raw_data = joblib.load(open(os.path.join(data_path,'stationary_sound_train.joblib'),'rb'))
    accel_raw_data = torch.from_numpy(np.concatenate(accel_raw_data)).to(device)
    sound_raw_data = torch.from_numpy(np.concatenate(sound_raw_data)).to(device)

In [34]:
tra = transfer_f.permute([1,2,0])
with ThreadPoolExecutor() as pool:
    a = torch.stack(list(pool.map(stft, tra)))

/root/anaconda3/envs/1.7/lib/python3.8/site-packages/torch/functional.py:515: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore


In [41]:
accel_

torch.Size([8, 8, 128])

In [38]:
def conv_with_S_freq(signal, S_data, config, device=torch.device('cpu')):
    # S_data(K, M, n_dims, Ls + 1)
    if config.ema:
        signal = ema(signal, n=2)
    
    Ls = S_data.size(1)
    K = S_data.size(-1)
    signal = padding(signal, Ls)
    if signal.size(1) != K:
        signal = signal.transpose(1,2)
    
    out = F.conv1d(signal, S_data.permute([2,0,1]).type(signal.dtype)).transpose(1,2)[:,:-1,:]
    
    return out 

In [42]:
def conv_freq(S_data, config, device):
    def _conv_freq(signal):
        '''()'''
        return conv_with_S_freq(signal, S_data, config, device)
    return _conv_freq
fileter = conv_freq(tra, config, device)

In [35]:
with ThreadPoolExecutor() as pool:
    a = torch.stack(list(pool.map(fileter, )))

torch.Size([8, 8, 2, 129])